# FullControl 1-minute demo

#### run all cells in this notebook in order (keep pressing shift+enter)

this quick demo shows how a design can be created with a list of points for nozzle movement with or without extrusion

the design is visually previewed, then gcode is created for a specific printer and saved to a file

for more information, see the [FullControl overview notebook](overview.ipynb)

<*this document is a jupyter notebook - if they're new to you, check out how they work:
[link](https://www.google.com/search?q=ipynb+tutorial),
[link](https://jupyter.org/try-jupyter/retro/notebooks/?path=notebooks/Intro.ipynb),
[link](https://colab.research.google.com/)*>

*run all cells in this notebook in order (keep pressing shift+enter)*

#### first, import fullcontrol to have access to its capabilities

In [1]:
import fullcontrol as fc

#### create and preview a design (a list of steps telling the printer what to do)

In [2]:
# # create an empty list called steps
# steps=[]
# # add points to the list
# steps.append(fc.Point(x=40,y=40,z=0.2))
# steps.append(fc.Point(x=50,y=50))
# steps.append(fc.Point(x=60,y=40))
# # turn the extruder on or off
# steps.append(fc.Extruder(on=False))
# steps.append(fc.Point(x=40,y=40,z=0.4))
# steps.append(fc.Extruder(on=True))
# steps.append(fc.Point(x=50,y=50))
# steps.append(fc.Point(x=60,y=40))
# # transform the design into a plot
# fc.transform(steps, 'plot')

In [3]:
# create an empty list called steps
steps=[]
# add points to the list
steps.append(fc.Point(x=0,y=0,z=0.2))
steps.append(fc.Point(x=0,y=50))
steps.append(fc.Point(x=50,y=50))
steps.append(fc.Point(x=50,y=0))
steps.append(fc.Point(x=0,y=0))
# turn the extruder on or off
steps.append(fc.Extruder(on=False))
# transform the design into a plot
fc.transform(steps, 'plot')

    - use fc.transform(..., controls=fc.PlotControls(style='tube') to disable this message or style='line' for a simpler line preview

fc.transform guidance tips are being written to screen if any potential issues are found - hide tips with fc.transform(..., show_tips=False)
tip: set initial `extrusion_width` and `extrusion_height` in the initialization_data to ensure the preview is correct:
   - `fc.transform(..., controls=fc.PlotControls(initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))`



In [24]:
import math
import fullcontrol as fc

# ========= Editable Parameters =========
z = 0.2
xmin, ymin = 20.0, 20.0
xmax, ymax = 70.0, 70.0

infill_angles = [45]   # try [0], [90], [45], or [45,-45] (crosshatch)
infill_ratio  = 0.95        # 25% infill (0.05..0.9 recommended)
line_width    = 1.75        # mm
layer_height  = 0.3         # mm
perimeter_shells = 3        # number of outer shells
infill_overlap_frac = 0.15     # 与内层壳线的搭接比例（按线宽），0~0.3 常用
infill_gap = 0.0               # 若想留缝而非搭接，可设为正值（mm）
travel_speed = 3600
print_speed  = 1200
# ======================================

def clamp(v, lo, hi): return max(lo, min(hi, v))

def add_perimeter_rect(steps, xmin, ymin, xmax, ymax, z, shells=1, w=0.4, speed_print=1200):
    """Adds outward->inward rectangles as perimeter shells."""
    for s in range(shells):
        inset = s * w/2
        x0, y0 = xmin + inset, ymin + inset
        x1, y1 = xmax - inset, ymax - inset

        # --- travel to this shell's start with extrusion OFF ---
        steps.append(fc.Extruder(on=False))
        steps.append(fc.travel_to(fc.Point(x=x0, y=y0, z=z)))

        # --- print this shell loop ---
        steps.append(fc.Extruder(on=True))
        steps.append(fc.Point(x=x1, y=y0, z=z))
        steps.append(fc.Point(x=x1, y=y1, z=z))
        steps.append(fc.Point(x=x0, y=y1, z=z))
        steps.append(fc.Point(x=x0, y=y0, z=z))  # close loop

        # leave extruder OFF for the travel to the next shell
        steps.append(fc.Extruder(on=False))

def _intersections_line_rect_y_eq_x_plus_c(xmin, ymin, xmax, ymax, c):
    """Line: y = x + c. Return list of points inside the rectangle."""
    pts = []
    # x = xmin / xmax
    y = xmin + c
    if ymin <= y <= ymax: pts.append((xmin, y))
    y = xmax + c
    if ymin <= y <= ymax: pts.append((xmax, y))
    # y = ymin / ymax
    x = ymin - c
    if xmin <= x <= xmax: pts.append((x, ymin))
    x = ymax - c
    if xmin <= x <= xmax: pts.append((x, ymax))
    # dedupe
    uniq = []
    seen = set()
    for p in pts:
        key = (round(p[0], 6), round(p[1], 6))
        if key not in seen:
            uniq.append(p); seen.add(key)
    return uniq

def _intersections_line_rect_y_eq_negx_plus_c(xmin, ymin, xmax, ymax, c):
    """Line: y = -x + c. Return list of points inside the rectangle."""
    pts = []
    # x = xmin / xmax
    y = -xmin + c
    if ymin <= y <= ymax: pts.append((xmin, y))
    y = -xmax + c
    if ymin <= y <= ymax: pts.append((xmax, y))
    # y = ymin / ymax
    x = -ymin + c
    if xmin <= x <= xmax: pts.append((x, ymin))
    x = -ymax + c
    if xmin <= x <= xmax: pts.append((x, ymax))
    # dedupe
    uniq = []
    seen = set()
    for p in pts:
        key = (round(p[0], 6), round(p[1], 6))
        if key not in seen:
            uniq.append(p); seen.add(key)
    return uniq

def add_infill_rect(steps, xmin, ymin, xmax, ymax, z,
                    angles, ratio, w, inset=0.2, alt_dir=True):
    """
    Rectilinear infill for a rectangle at height z.
    angles: list of {0, 90, 45, -45}
    ratio:  areal fraction (0..1)
    w:      line width (mm)
    inset:  keep infill this far inside the shell (mm)
    """
    # sanitize
    ratio = clamp(ratio, 0.02, 0.95)
    angles = list(angles)
    for a in angles:
        if a not in (0, 90, 45, -45):
            raise NotImplementedError("Supported angles are 0, 90, 45, -45.")
    n = max(1, len(angles))

    # spacing (perpendicular pitch) to hit target ratio, approx: rho ≈ n * w / pitch
    pitch = (n * w) / ratio

    # inset box (stay away from perimeter)
    x0 = xmin + inset
    y0 = ymin + inset
    x1 = xmax - inset
    y1 = ymax - inset

    # For alternating travel direction
    lr = True

    for a in angles:
        steps.append(fc.Extruder(on=False))  # safe travel before starting this pass
        if a == 0:
            # horizontal lines: step in +Y
            y = y0
            while y <= y1 + 1e-9:
                # endpoints for the segment
                xA, xB = x0, x1
                if alt_dir and not lr:
                    xA, xB = xB, xA
                # travel to start
                steps.append(fc.Point(x=xA, y=y, z=z))
                # print the segment
                steps.append(fc.Extruder(on=True))
                steps.append(fc.Point(x=xB, y=y, z=z))
                # steps.append(fc.Extruder(on=False))
                lr = not lr
                y += pitch

        elif a == 90:
            # vertical lines: step in +X
            x = x0
            while x <= x1 + 1e-9:
                yA, yB = y0, y1
                if alt_dir and not lr:
                    yA, yB = yB, yA
                steps.append(fc.Point(x=x, y=yA, z=z))
                steps.append(fc.Extruder(on=True))
                steps.append(fc.Point(x=x, y=yB, z=z))
                # steps.append(fc.Extruder(on=False))
                lr = not lr
                x += pitch

        elif a == 45:
            # lines y = x + c ; perpendicular spacing Δc = pitch*sqrt(2)
            corners = [(x0,y0),(x0,y1),(x1,y0),(x1,y1)]
            c_vals = [y - x for (x,y) in corners]
            c_min, c_max = min(c_vals), max(c_vals)
            dc = pitch * math.sqrt(2.0)
            c = c_min
            while c <= c_max + 1e-9:
                pts = _intersections_line_rect_y_eq_x_plus_c(x0,y0,x1,y1,c)
                if len(pts) >= 2:
                    # take farthest two along direction (x+y grows along slope +1)
                    pts_sorted = sorted(pts, key=lambda p: p[0]+p[1])
                    pA, pB = pts_sorted[0], pts_sorted[-1]
                    if alt_dir and not lr:
                        pA, pB = pB, pA
                    steps.append(fc.Point(x=pA[0], y=pA[1], z=z))
                    steps.append(fc.Extruder(on=True))
                    steps.append(fc.Point(x=pB[0], y=pB[1], z=z))
                    # steps.append(fc.Extruder(on=False))
                    lr = not lr
                c += dc

        elif a == -45:
            # lines y = -x + c ; perpendicular spacing Δc = pitch*sqrt(2)
            corners = [(x0,y0),(x0,y1),(x1,y0),(x1,y1)]
            c_vals = [y + x for (x,y) in corners]  # since c = y - (-x) = y + x
            c_min, c_max = min(c_vals), max(c_vals)
            dc = pitch * math.sqrt(2.0)
            c = c_min
            while c <= c_max + 1e-9:
                pts = _intersections_line_rect_y_eq_negx_plus_c(x0,y0,x1,y1,c)
                if len(pts) >= 2:
                    # along slope -1, use (y - x) or (-x + y) as progress; choose x - y for variety
                    pts_sorted = sorted(pts, key=lambda p: p[0]-p[1])
                    pA, pB = pts_sorted[0], pts_sorted[-1]
                    if alt_dir and not lr:
                        pA, pB = pB, pA
                    steps.append(fc.Point(x=pA[0], y=pA[1], z=z))
                    steps.append(fc.Extruder(on=True))
                    steps.append(fc.Point(x=pB[0], y=pB[1], z=z))
                    # steps.append(fc.Extruder(on=False))
                    lr = not lr
                c += dc

# ----------------- Build the program -----------------
steps = []
steps.append(fc.Printer(print_speed=print_speed, travel_speed=travel_speed))
steps.append(fc.ExtrusionGeometry(area_model='rectangle', width=line_width, height=layer_height))

# Perimeters
add_perimeter_rect(steps, xmin, ymin, xmax, ymax, z,
                   shells=perimeter_shells, w=line_width)

inset_for_infill = max(
    0.0,
    (perimeter_shells - infill_overlap_frac) * line_width/2 + infill_gap
)

# Infill (keep slightly inside the last shell)
add_infill_rect(
    steps, xmin, ymin, xmax, ymax, z,
    angles=infill_angles,
    ratio=infill_ratio,
    w=line_width,
    inset=inset_for_infill,   # <<< 关键：基于“内层壳线”
    alt_dir=True
)

# Visualize
fc.transform(steps, 'plot', fc.PlotControls(style='line'))

# If you want G-code:
# gcode = fc.transform(steps, 'gcode', fc.GcodeControls(printer_name='generic', save_as='rect_infill_var'))
# print(gcode[:400])


warning - the list of steps should be a 1D list of fullcontrol class instances, it currently includes a 'list'
   - fc.flatten() is being used to convert the design to a 1D list


#### set filename, printer and print settings

In [25]:
filename = 'my_design'
printer = 'prusa_i3' 
# printer options: generic, ultimaker2plus, prusa_i3, ender_3, cr_10, bambulab_x1, toolchanger_T0, toolchanger_T1, toolchanger_T2, toolchanger_T3
print_settings = {'extrusion_width': 1.75,'extrusion_height': 0.2, 'nozzle_temp': 210, 'bed_temp': 40, 'fan_percent': 100}
# 'extrusion_width' and 'extrusion_height' are the width and height of the printed line)

#### save gcode file to the same directory as this notebook

do not edit this line of code - it uses values defined in the previous code cells

make sure you execute the previous cells before running this one

In [26]:
fc.transform(steps, 'gcode', fc.GcodeControls(printer_name=printer, save_as=filename, initialization_data=print_settings))

warning - the list of steps should be a 1D list of fullcontrol class instances, it currently includes a 'list'
   - fc.flatten() is being used to convert the design to a 1D list


"; Time to print!!!!!\n; GCode created with FullControl - tell us what you're printing!\n; info@fullcontrol.xyz or tag FullControlXYZ on Twitter/Instagram/LinkedIn/Reddit/TikTok \n\nM140 S40 ; set bed temp and continue\nM104 S210 ; set hotend temp and continue\nM190 S40 ; set bed temp and wait\nM109 S210  ; set hotend temp and wait\nG28 ; home axes ; ; including mesh bed level\nG90 ; absolute coordinates\nG21 ; set units to millimeters\nM83 ; relative extrusion\nM106 S255 ; set fan speed\nM220 S100 ; set speed factor override percentage\nM221 S100 ; set extrude factor override percentage\nG0 F8000 X5 Y5 Z10\nG1 F250 E20.7876\nG0 F250 Z50\nG0 F8000 X10 Y10 Z0.3\n;-----\n; END OF STARTING PROCEDURE\n;-----\n\n;-----\n; START OF PRIMER PROCEDURE\n;-----\nG0 F8000 Y12 Z0.2\nG1 F1000 X110 E14.551309\nG1 Y14 E0.291026\nG1 X10 E14.551309\nG1 Y16 E0.291026\nG1 X20 E1.455131\nG1 Y20 E0.582052\n;-----\n; END OF PRIMER PROCEDURE\n;-----\n\nG1 F1200 X70 E10.913482\nG1 Y70 E10.913482\nG1 X20 E10.91

#### get creative!

check out [other tutorials](contents.ipynb) to see how to create designs like this gear/thread example with just one line of code

In [7]:
steps = [fc.polar_to_point(centre=fc.Point(x=0, y=0, z=i*0.005), radius=10, angle=i*4.321) for i in range(1000)]
fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True, zoom=0.7))

    - use fc.transform(..., controls=fc.PlotControls(style='tube') to disable this message or style='line' for a simpler line preview

fc.transform guidance tips are being written to screen if any potential issues are found - hide tips with fc.transform(..., show_tips=False)
tip: set initial `extrusion_width` and `extrusion_height` in the initialization_data to ensure the preview is correct:
   - `fc.transform(..., controls=fc.PlotControls(initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))`



#### random mesh example

In [8]:
from math import tau
from random import random
steps=[fc.polar_to_point(centre=fc.Point(x=0, y=0, z=i*0.001), radius=10+5*random(), angle=i*tau/13.8) for i in range(4000)]
fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True, zoom=0.7))

    - use fc.transform(..., controls=fc.PlotControls(style='tube') to disable this message or style='line' for a simpler line preview

fc.transform guidance tips are being written to screen if any potential issues are found - hide tips with fc.transform(..., show_tips=False)
tip: set initial `extrusion_width` and `extrusion_height` in the initialization_data to ensure the preview is correct:
   - `fc.transform(..., controls=fc.PlotControls(initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))`

